In [77]:

import numpy as np
import pandas as pd


# paths to the dataset with encoding B specified
train_df = pd.read_csv('/kaggle/input/ml-hackathon-ec-campus-set-3-alt/train/text.csv', encoding='ISO-8859-1')
test_df = pd.read_csv('/kaggle/input/ml-hackathon-ec-campus-set-3-alt/test/text.csv', encoding='ISO-8859-1')

# Print the first five rows dataset
print(train_df.head())
print(test_df.head())

# Check the shape of the datasets
print("Train_shape:", train_df.shape)
print("Test_shape:", test_df.shape)


   Sr No.                                      Utterance          Speaker  \
0       4  So lets talk a little bit about your duties.  The Interviewer   
1      18       No, I-I-I-I don't, I actually don't know           Rachel   
2      40       He was with her when he wrote this poem.           Phoebe   
3      53                                            Hi.           Rachel   
4      69                            Which part exactly?             Joey   

  Sentiment  Dialogue_ID  Utterance_ID  Season  Episode     StartTime  \
0   neutral            0             3       8       21  00:16:26,820   
1  negative            1             3       9       23  00:36:49,290   
2   neutral            3             3       3       12  00:10:21,078   
3   neutral            4             6       3       11   0:20:30,873   
4   neutral            6             3       8       22   0:12:31,655   

        EndTime  
0  00:16:29,572  
1  00:36:51,791  
2  00:10:23,496  
3   0:20:31,135  
4   0:12

In [78]:
#Load and Link Data with Video Paths"
import os
import pandas as pd

# Load training and testing datasets with specified encoding to handle special characters
train_df = pd.read_csv('/kaggle/input/ml-hackathon-ec-campus-set-3-alt/train/text.csv', encoding='ISO-8859-1')
test_df = pd.read_csv('/kaggle/input/ml-hackathon-ec-campus-set-3-alt/test/text.csv', encoding='ISO-8859-1')

#  Define directory locations for training and testing video files
train_video_friends = '/kaggle/input/ml-hackathon-ec-campus-set-3-alt/train/videos'
test_video_friends = '/kaggle/input/ml-hackathon-ec-campus-set-3-alt/test/videos'

#  Function to construct the path for each video clip based on dialogue and utterance IDs
def construct_video_path(row, video_path):
    dialogue_id = row['Dialogue_ID']
    utterance_id = row['Utterance_ID']
    filename = f"dia{dialogue_id}_utt{utterance_id}.mp4"
    return os.path.join(video_path, filename)

# Apply the function to each row to create a new column for video file paths
train_df['video_clip_path'] = train_df.apply(construct_video_path, axis=1, video_path=train_video_friends)
test_df['video_clip_path'] = test_df.apply(construct_video_path, axis=1, video_path=test_video_friends)

# Display sample rows to verify the video paths are added accurately
print(train_df[['Dialogue_ID', 'Utterance_ID', 'video_clip_path']].head())
print(test_df[['Dialogue_ID', 'Utterance_ID', 'video_clip_path']].head())

   Dialogue_ID  Utterance_ID  \
0            0             3   
1            1             3   
2            3             3   
3            4             6   
4            6             3   

                                     video_clip_path  
0  /kaggle/input/ml-hackathon-ec-campus-set-3-alt...  
1  /kaggle/input/ml-hackathon-ec-campus-set-3-alt...  
2  /kaggle/input/ml-hackathon-ec-campus-set-3-alt...  
3  /kaggle/input/ml-hackathon-ec-campus-set-3-alt...  
4  /kaggle/input/ml-hackathon-ec-campus-set-3-alt...  
   Dialogue_ID  Utterance_ID  \
0            5             8   
1            7             6   
2           12             1   
3           12             9   
4           13             5   

                                     video_clip_path  
0  /kaggle/input/ml-hackathon-ec-campus-set-3-alt...  
1  /kaggle/input/ml-hackathon-ec-campus-set-3-alt...  
2  /kaggle/input/ml-hackathon-ec-campus-set-3-alt...  
3  /kaggle/input/ml-hackathon-ec-campus-set-3-alt...  
4  /kaggl

In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer for text feature extraction (limited to 500 features)
tfidf = TfidfVectorizer(max_features=500)

# Apply TF-IDF transformation on 'Utterance' column to obtain text-based features
train_friends_text_features = tfidf.fit_transform(train_df['Utterance']).toarray()
test_friends_text_features = tfidf.transform(test_df['Utterance']).toarray()

# Merge the generated text features with the original DataFrames for future use
train_df = pd.concat([train_df, pd.DataFrame(train_friends_text_features)], axis=1)
test_df = pd.concat([test_df, pd.DataFrame(test_friends_text_features)], axis=1)

# Display the first few rows to confirm the features were correctly appended
print(train_df.head())
print(test_df.head())


   Sr No.                                      Utterance          Speaker  \
0       4  So lets talk a little bit about your duties.  The Interviewer   
1      18       No, I-I-I-I don't, I actually don't know           Rachel   
2      40       He was with her when he wrote this poem.           Phoebe   
3      53                                            Hi.           Rachel   
4      69                            Which part exactly?             Joey   

  Sentiment  Dialogue_ID  Utterance_ID  Season  Episode     StartTime  \
0   neutral            0             3       8       21  00:16:26,820   
1  negative            1             3       9       23  00:36:49,290   
2   neutral            3             3       3       12  00:10:21,078   
3   neutral            4             6       3       11   0:20:30,873   
4   neutral            6             3       8       22   0:12:31,655   

        EndTime  ...  490  491  492  493  494  495  496  497       498  499  
0  00:16:29,572  ...

In [80]:
import cv2
import numpy as np
import pandas as pd
import cupy as cp  # GPU acceleration library
from tqdm import tqdm  # To monitor progress

# Function to extract features from video frames, using GPU for faster computation
def extract_video_features_on_gpu(video_path):
    frame_features = []
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Resize frame to 224x224 using GPU
        frame_gpu = cp.array(cv2.resize(frame, (224, 224)))  # Convert to GPU array using cupy
        # Flatten the frame to a 1D vector and append to feature list
        frame_features.append(cp.asnumpy(frame_gpu).flatten())  # Transfer back to CPU for further processing

    cap.release()
    
    # Return the mean of the features (alternative to using deep learning features)
    return np.mean(frame_features, axis=0) if frame_features else np.zeros((224 * 224 * 3,))

# Initialize list to store video features for the training set
train_video_features_list = []

# Using tqdm to display a progress bar while processing video files
for video_path in tqdm(train_df['video_clip_path']):
    train_video_features_list.append(extract_video_features_on_gpu(video_path))

# Convert the extracted features to a DataFrame for easy integration
train_video_features_df = pd.DataFrame(train_video_features_list)

# Merge the video features with the original DataFrame (reset index for alignment)
train_df = pd.concat([train_df.reset_index(drop=True), train_video_features_df], axis=1)

# Display the DataFrame with the newly added features
print(train_df.head())


100%|██████████| 1000/1000 [03:37<00:00,  4.59it/s]


   Sr No.                                      Utterance          Speaker  \
0       4  So lets talk a little bit about your duties.  The Interviewer   
1      18       No, I-I-I-I don't, I actually don't know           Rachel   
2      40       He was with her when he wrote this poem.           Phoebe   
3      53                                            Hi.           Rachel   
4      69                            Which part exactly?             Joey   

  Sentiment  Dialogue_ID  Utterance_ID  Season  Episode     StartTime  \
0   neutral            0             3       8       21  00:16:26,820   
1  negative            1             3       9       23  00:36:49,290   
2   neutral            3             3       3       12  00:10:21,078   
3   neutral            4             6       3       11   0:20:30,873   
4   neutral            6             3       8       22   0:12:31,655   

        EndTime  ...      150518     150519     150520      150521  \
0  00:16:29,572  ...   37.80

In [81]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Combine the text-based features and video-based features for early fusion
text_features_combined = np.hstack((train_text_features, train_video_features))
sentiment_labels = train_df['Sentiment']

# Split the dataset into training and validation sets (80% train, 20% validation)
X_train_set, X_validation_set, y_train_set, y_validation_set = train_test_split(text_features_combined, sentiment_labels, test_size=0.2, random_state=42)

# Initialize a RandomForestClassifier
classifier_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model to the training data
classifier_model.fit(X_train_set, y_train_set)

# Make predictions on the validation set
predicted_labels = classifier_model.predict(X_validation_set)

# Display classification metrics for the model
print(classification_report(y_validation_set, predicted_labels))


              precision    recall  f1-score   support

    negative       0.50      0.15      0.23        61
     neutral       0.50      0.90      0.64        97
    positive       0.25      0.05      0.08        42

    accuracy                           0.49       200
   macro avg       0.42      0.36      0.32       200
weighted avg       0.45      0.49      0.40       200



In [82]:
from sklearn.metrics import accuracy_score, classification_report

# Evaluate accuracy for the model
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Model_Accuracy:", accuracy*100)

Model_Accuracy: 47.0


In [84]:
# Generate features for the test data (video features)
test_video_features = np.array([extract_video_features(path) for path in test_df['video_clip_path']])

# Combine the test data's text and video features for prediction
X_test = np.hstack((test_friends_text_features, test_video_features))

# Predict on test data
Tests_Predictions = model.predict(X_test)

# Create submission DataFrame with "Sr No." from 1 to 100
submission_df = pd.DataFrame({
    'Sr No.': test_df['Sr No.'],
    'Sentiment': Tests_Predictions
})

# Save predictions to CSV for submission
submission_df.to_csv("submission.csv", index=False)


